In this notebook, we will see how we can make use nnUnet to make masks for this competition. 

What is [nnUnet](https://github.com/MIC-DKFZ/nnUNet)?
>nnU-Net is the first segmentation method that is designed to deal with the dataset diversity found in the domain (medical). It condenses and automates the key decisions for designing a successful segmentation pipeline for any given dataset. 
nnUnet was published in [Nature](https://www.nature.com/articles/s41592-020-01008-z)

Abstract from nature:
>Biomedical imaging is a driver of scientific discovery and a core component of medical care and is being stimulated by the field of deep learning. While semantic segmentation algorithms enable image analysis and quantification in many applications, the design of respective specialized solutions is non-trivial and highly dependent on dataset properties and hardware conditions. We developed nnU-Net, a deep learning-based segmentation method that automatically configures itself, including preprocessing, network architecture, training and post-processing for any new task. The key design choices in this process are modeled as a set of fixed parameters, interdependent rules and empirical decisions. Without manual intervention, nnU-Net surpasses most existing approaches, including highly specialized solutions on 23 public datasets used in international biomedical segmentation competitions. We make nnU-Net publicly available as an out-of-the-box tool, rendering state-of-the-art segmentation accessible to a broad audience by requiring neither expert knowledge nor computing resources beyond standard network training.

nnUnet lead the [Medical Decathlon competition](http://medicaldecathlon.com/results/).  

We will make use of nnUnet's Task01-Brain Tumouur Segmentation pretrained model to generate masks. 

We will follow the following steps
1. Select 5 random patients
2. Resample the images using SimpleITK and SIR24 dataset - I have a [notebook](https://www.kaggle.com/marshath/btrg-dicom-to-nifti-using-sri24-to-resample) completely for this
3. Git clone and set up nnUnet - This [notebook](https://github.com/prateekgupta891/nnUNet/blob/master/nnunetmec2020.ipynb) here was my light
4. Prepare the data as per nnUnet's requirement
5. Make masks!

# 0. Imports and paths

In [ ]:
import os
from fastai.vision.all import *
import SimpleITK as sitk
import shutil

In [ ]:
path = Path('../input/rsna-miccai-brain-tumor-radiogenomic-classification/train')

In [ ]:
!mkdir t1w_nifti
!mkdir t1wce_nifti
!mkdir t2w_nifti
!mkdir flair_nifti

# 1. Select 5 random patients 

In [ ]:
samples = [Path('../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/00002'),
           Path('../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/00194'),
           Path('../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/00094'),
           Path('../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/00734'),
           Path('../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/00526')]

In [ ]:
path_train_t2w, path_train_t1wce,path_train_t1w,path_train_flair = [],[],[],[]
for each in samples:
    path_train_t2w.append(each.ls()[0])
    path_train_t1wce.append(each.ls()[1])
    path_train_t1w.append(each.ls()[2])
    path_train_flair.append(each.ls()[3])

In [ ]:
def dicom2nifti(image_dir, out_dir, save=True):
    "given a dicom directory, loads them into single file and can save it as .nii file"
    reader = sitk.ImageSeriesReader()
    reader.LoadPrivateTagsOn()
    filenamesDICOM = reader.GetGDCMSeriesFileNames(str(image_dir))
    reader.SetFileNames(filenamesDICOM)
    img = reader.Execute()
    img = sitk.Cast(img, sitk.sitkFloat32)
    
    if save:
        sitk.WriteImage(img, f'{out_dir}/{image_dir.parent.name}.nii')
    else:
        return img

Let's convert the images into nifti

In [ ]:
for fn in path_train_t1w: dicom2nifti(fn, "/kaggle/working/t1w_nifti/")
for fn in path_train_t1wce: dicom2nifti(fn, "/kaggle/working/t1wce_nifti/")
for fn in path_train_t2w: dicom2nifti(fn, "/kaggle/working/t2w_nifti/")
for fn in path_train_flair: dicom2nifti(fn, "/kaggle/working/flair_nifti/")

Let's take a look at the images

In [ ]:
def get_array(fn):
    "opens .nii file and return the array"
    img = sitk.ReadImage(str(fn))
    imgd = sitk.GetArrayFromImage(img)
    return imgd

def plot_slice(imgd, sli):
    "given an image of shape slices x height x width, plots a slice"
    plt.imshow(imgd[sli], cmap='gray')
    plt.axis('off')
    
def get_array_plot(fn, sli):
    imgd = get_array(fn)
    plot_slice(imgd, sli)

In [ ]:
samp_pat = '00094'

In [ ]:
get_array_plot(Path(f'./t1w_nifti/{samp_pat}.nii'), 20)

In [ ]:
get_array_plot(Path(f'./t1wce_nifti/{samp_pat}.nii'), 45)

In [ ]:
get_array_plot(Path(f'./t2w_nifti/{samp_pat}'), 120)

In [ ]:
get_array_plot(Path(f'./flair_nifti/{samp_pat}'), 80)

### 2. Resample the images using SimpleITK and SIR24 dataset

As we can see, for orientations are different across the different modalities. Let's resample using SRI24 as the template. 

In [ ]:
def resample_nifti(image_dir, ref_image, fn, save=True):
    "resample using a reference image"

    image = sitk.ReadImage(str(image_dir), sitk.sitkFloat32)
    
    initial_transform = sitk.CenteredTransformInitializer(ref_image, 
                                                          image, 
                                                          sitk.Euler3DTransform(), 
                                                          sitk.CenteredTransformInitializerFilter.GEOMETRY)

    resampler = sitk.ResampleImageFilter()
    resampler.SetReferenceImage(ref_image)
    resampler.SetInterpolator(sitk.sitkLinear)
    resampler.SetTransform(initial_transform)
    resampler.SetOutputSpacing(ref_image.GetSpacing())
    resampler.SetSize((ref_image.GetSize()))
    resampler.SetOutputDirection(ref_image.GetDirection())
    resampler.SetOutputOrigin(ref_image.GetOrigin())
    resampler.SetDefaultPixelValue(image.GetPixelIDValue())
    resamped_image = resampler.Execute(image)
    
    if save:
        sitk.WriteImage(resamped_image, fn)

    return resamped_image

In [ ]:
ref_image = sitk.ReadImage('../input/sri24-dataset/sri24/spgr.nii', sitk.sitkFloat32)

In [ ]:
!mkdir t1w_resample
!mkdir t1wce_resample
!mkdir t2w_resample
!mkdir flair_resample

In [ ]:
for fn in Path("./t1w_nifti").ls():
    pat_id = str(fn).split('/')[-1].split('.')[0]
    final_fn = f"/kaggle/working/t1w_resample/{pat_id}_0001.nii.gz"
    resample_nifti(fn, ref_image, final_fn, True)
    os.remove(str(fn))

In [ ]:
for fn in Path("./t1wce_nifti").ls():
    pat_id = str(fn).split('/')[-1].split('.')[0]
    final_fn = f"/kaggle/working/t1wce_resample/{pat_id}_0002.nii.gz"
    resample_nifti(fn, ref_image, final_fn, True)
    os.remove(str(fn))

In [ ]:
for fn in Path("./t2w_nifti").ls():
    pat_id = str(fn).split('/')[-1].split('.')[0]
    final_fn = f"/kaggle/working/t2w_resample/{pat_id}_0003.nii.gz"
    resample_nifti(fn, ref_image, final_fn, True)
    os.remove(str(fn))

In [ ]:
for fn in Path("./flair_nifti").ls():
    pat_id = str(fn).split('/')[-1].split('.')[0]
    final_fn = f"/kaggle/working/flair_resample/{pat_id}_0000.nii.gz"
    resample_nifti(fn, ref_image, final_fn, True)
    os.remove(str(fn))

In [ ]:
get_array_plot(Path(f'/kaggle/working/t1w_resample/{samp_pat}_0001.nii.gz'), 85)

In [ ]:
get_array_plot(Path(f'/kaggle/working/t1wce_resample/{samp_pat}_0002.nii.gz'), 85)

In [ ]:
get_array_plot(Path(f'/kaggle/working/t2w_resample/{samp_pat}_0003.nii.gz'), 85)

In [ ]:
get_array_plot(Path(f'/kaggle/working/flair_resample/{samp_pat}_0000.nii.gz'), 85)

In [ ]:
!rm -rf t1w_nifti
!rm -rf t1wce_nifti
!rm -rf t2w_nifti
!rm -rf flair_nifti

Looks like we have resampled them. If you noticed, we added four digits to the file names. This is because nnUnet (Task01 - Brain Tumour) requires all four modalities 

`0000 - flair, 0001 - T1w, 0002 - T1wce, 0003 - T2w`

# 3. Git clone and set up nnUnet

`labels: {'0': 'background',  '1': 'edema', '2': 'non-enhancing tumor', '3': 'enhancing tumour'}` - following is the labels 

In [ ]:
import os
base_dir = '/kaggle/working/'
os.chdir(base_dir)

In [ ]:
!git clone https://github.com/MIC-DKFZ/nnUNet.git 
!git clone https://github.com/NVIDIA/apex

In [ ]:
respository_dir = os.path.join(base_dir,'nnUNet')
os.chdir(respository_dir)

!pip install -e .
#(optional installation)
!pip install --upgrade git+https://github.com/nanohanno/hiddenlayer.git@bugfix/get_trace_graph#egg=hiddenlayer

os.chdir(base_dir)

**Note** Restart the notebook

In [ ]:
#import IPython
#IPython.Application.instance().kernel.do_shutdown(True)

In [ ]:
import os
from fastai.vision.all import *
import SimpleITK as sitk
import shutil

base_dir = '/kaggle/working/'
os.chdir(base_dir)

## Dataset Folder Structure

In [ ]:
def make_if_dont_exist(folder_path,overwrite=False):
    """
    creates a folder if it does not exists
    input: 
    folder_path : relative path of the folder which needs to be created
    over_write :(default: False) if True overwrite the existing folder 
    """
    if os.path.exists(folder_path):
        
        if not overwrite:
            print(f'{folder_path} exists.')
        else:
            print(f"{folder_path} overwritten")
            shutil.rmtree(folder_path)
            os.makedirs(folder_path)

    else:
        os.makedirs(folder_path)
        print(f"{folder_path} created!")

In [ ]:
task_name = 'Task001_BrainTumour' #change here for different task name
nnunet_dir = "nnUNet/nnunet/nnUNet_raw_data_base/nnUNet_raw_data"
task_folder_name = os.path.join(nnunet_dir,task_name)
train_image_dir = os.path.join(task_folder_name,'imagesTr')
train_label_dir = os.path.join(task_folder_name,'labelsTr')
test_dir = os.path.join(task_folder_name,'imagesTs')
main_dir = os.path.join(base_dir,'nnUNet/nnunet')

In [ ]:
make_if_dont_exist(task_folder_name,overwrite = False)
make_if_dont_exist(train_image_dir)
make_if_dont_exist(train_label_dir)
make_if_dont_exist(test_dir,overwrite= False)
make_if_dont_exist(os.path.join(main_dir,'nnunet_trained_models'))

## Environment Veriables

In [ ]:
os.environ['nnUNet_raw_data_base'] = os.path.join(main_dir,'nnUNet_raw_data_base')
os.environ['nnUNet_preprocessed'] = os.path.join(main_dir,'preprocessed')
os.environ['RESULTS_FOLDER'] = os.path.join(main_dir,'nnUNet_trained_models')

## Download the pretrained model

In [ ]:
#!nnUNet_print_available_pretrained_models
!nnUNet_download_pretrained_model Task001_BrainTumour

# 4. Prepare the data as per nnUnet's requirement

In [ ]:
path_ts = '/kaggle/working/nnUNet/nnunet/nnUNet_raw_data_base/nnUNet_raw_data/Task001_BrainTumour/imagesTs'

In [ ]:
for fn in Path("./t1w_resample").ls(): 
    pat_id = fn.name.split('.')[0].split('_')[0]
    shutil.copy(f'/kaggle/working/t1w_resample/{pat_id}_0001.nii.gz', f'{path_ts}')
    shutil.copy(f'/kaggle/working/t1wce_resample/{pat_id}_0002.nii.gz', f'{path_ts}/{pat_id}_0002.nii.gz')
    shutil.copy(f'/kaggle/working/t2w_resample/{pat_id}_0003.nii.gz', f'{path_ts}/{pat_id}_0003.nii.gz')
    shutil.copy(f'/kaggle/working/flair_resample/{pat_id}_0000.nii.gz', f'{path_ts}/{pat_id}_0000.nii.gz')
    
    os.remove(f'/kaggle/working/t1w_resample/{pat_id}_0001.nii.gz')
    os.remove(f'/kaggle/working/t1wce_resample/{pat_id}_0002.nii.gz')
    os.remove(f'/kaggle/working/t2w_resample/{pat_id}_0003.nii.gz')
    os.remove(f'/kaggle/working/flair_resample/{pat_id}_0000.nii.gz')

# 5. Inference - Make Masks

In [ ]:
result_dir = os.path.join(main_dir,'nnUNet_Prediction_Results',task_name)
make_if_dont_exist(result_dir)

team_name = 'arshath'

#location where you want save your results, will be created if dont exist
os.chdir(main_dir)
!nnUNet_predict -i nnUNet_raw_data_base/nnUNet_raw_data/Task001_BrainTumour/imagesTs -o nnUNet_Prediction_Results/Task001_BrainTumour -t 001 -tr nnUNetTrainerV2 -m 3d_fullres --disable_tta 
os.chdir(base_dir)

## Finally let's check the masks

In [ ]:
import os
from fastai.vision.all import *
import SimpleITK as sitk
import shutil

In [ ]:
path_ts = '/kaggle/working/nnUNet/nnunet/nnUNet_raw_data_base/nnUNet_raw_data/Task001_BrainTumour/imagesTs'
path_lbl = '/kaggle/working/nnUNet/nnunet/nnUNet_Prediction_Results/Task001_BrainTumour'
samp_pat = '00094'

In [ ]:
def get_array(fn):
    "opens .nii file and return the array"
    img = sitk.ReadImage(str(fn))
    imgd = sitk.GetArrayFromImage(img)
    return imgd

def plot_slice(imgd, sli):
    "given an image of shape slices x height x width, plots a slice"
    plt.imshow(imgd[sli], cmap='gray')
    plt.axis('off')
    
def get_array_plot(fn, sli):
    imgd = get_array(fn)
    plot_slice(imgd, sli)

In [ ]:
get_array_plot(f'{path_ts}/{samp_pat}_0003.nii.gz', 85)

In [ ]:
get_array_plot(f'{path_lbl}/{samp_pat}.nii.gz', 85)